### Import Dependencies

In [1]:
#gym
import gym
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete
#python helpers
import numpy as np
import random
import os
#stable baselines
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

### Type of Spaces

In [4]:
Discrete(3).sample()

0

In [7]:
Box(0,1, shape=(3,3)).sample()

array([[0.95204407, 0.6718285 , 0.12941825],
       [0.7512171 , 0.17331198, 0.10886811],
       [0.06586929, 0.7030815 , 0.18170595]], dtype=float32)

In [10]:
Tuple((Discrete(3), Box(0,1, shape=(3,3)))).sample()

(1,
 array([[0.4981536 , 0.9019075 , 0.00146689],
        [0.740228  , 0.37211877, 0.942188  ],
        [0.08638076, 0.07152137, 0.05881456]], dtype=float32))

In [12]:
Dict({'height':Discrete(2), 'speed':Box(0,100,shape=(1,))})

Dict(height:Discrete(2), speed:Box([0.], [100.], (1,), float32))

In [14]:
MultiBinary(4).sample()

array([1, 0, 0, 1], dtype=int8)

In [19]:
MultiDiscrete([5,2,2,5]).sample()

array([2, 1, 0, 0])

### Building an Environment
- Build an agent to give us the best shower posible
- Randomly temperature
- 37 and 39 degrees

In [42]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        ##self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.observation_space = Box(low=0, high=100, shape=(1,))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
    
    def step(self, action):    
        #apply temp adjustment
        self.state += action -1
        
        #decrese shower time
        self.shower_length -= 1
        
        #calculate reward
        if self.state >=37 and self.state <=39:
            reward = 1
        else:
            reward = -1
        if self.shower_length <= 0:
            done = True
        else:
            done = False
        
        info = {}
        
        return self.state, reward, done, info
        
    def render(self):
        #implement viz
        pass
    def reset(self):
        self.state = np.array([38+random.randint(-3,3)]).astype(float)
        self.shower_length = 60
        return self.state

In [43]:
env = ShowerEnv()

In [44]:
env.observation_space.sample()

array([19.733774], dtype=float32)

In [45]:
env.action_space.sample()

0

### Test Environment

In [46]:
episodes =  5
for episode in range(1, episodes+1):
    obs =  env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-14
Episode:2 Score:-24
Episode:3 Score:-20
Episode:4 Score:-44
Episode:5 Score:14


### Train Model

In [49]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [52]:
modemodelrn(total_timesteps=400000)

Logging to Training/Logs/PPO_12
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -3.12    |
| time/              |          |
|    fps             | 1411     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -9.03       |
| time/                   |             |
|    fps                  | 1040        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.003213942 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.946      |
|    explained_variance   | 0.0602      

### Save Model

In [53]:
ppo_path = os.path.join('Training','Saved Models','PPO_Shower_Model')

In [54]:
model.save(ppo_path)

In [55]:
del model

In [56]:
model = PPO.load(ppo_path)

### Test Model

In [58]:
evaluate_policy(model, env, n_eval_episodes=5)

(-36.0, 48.0)